In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Angela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

df = pd.read_csv("tagged category data.csv", encoding = "ISO-8859-1")
# df.set_index('review_id', inplace=True)
df2 = pd.read_csv("small_annotation.csv", encoding = "ISO-8859-1")
# df2.set_index('review_id', inplace=True)
sentiment_df = pd.read_csv("sentiment annotation data.csv", encoding = "ISO-8859-1")
# sentiment_df.set_index('review_id', inplace=True)

In [3]:
#no stop word
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

df['text'] = df['text'].map(lambda com : clean_text(com))

In [4]:
#stop word filter
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text_2(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

df2['text'] = df2['text'].map(lambda com : clean_text_2(com))

In [5]:
df2.head()

,review_id,stars,text,food,ambience,service,price
0,EdLtoDFxxUiL-yaT3KNu4g,4,good environ order famou meatloaf larg littl b...,1,1,1,0
1,oqsViseLh2tpKxxV3bEAcQ,4,local favorit hole wall serv home cook food di...,1,0,1,0
2,6BnQwlxRn7ZuWdzninM9sQ,3,love chines food love mexican food wrong coupl...,1,1,1,0
3,TFfoC8UJzvQ96abwA7I9wQ,5,good clean place mani pedi came tax great job ...,1,0,1,1
4,slWKNylDacmBn7Dq5pbZcQ,1,eat place scare food worst servic trashi anti ...,1,1,1,0


In [6]:
sentiment_df['text'] = sentiment_df['text'].map(lambda com : clean_text_2(com))
sentiment_df.head()


,review_id,stars,text,food_pos,ambience_pos,service_pos,price_pos
0,yi0R0Ugj_xUx_Nek0-_Qig,5,went lunch steak sandwich delici caesar salad ...,1.0,NaN,1.0,1.0
1,fdiNeiN_hoCxCMy2wTRW9g,4,admit excit go tavolta food snob group friend ...,1.0,0.0,0.0,1.0
2,G7XHMxG0bx9oBJNECG4IFg,3,traci dessert big hong kong markham place year...,1.0,1.0,NaN,0.0
3,8e9HxxLjjqc9ez5ezzN7iQ,1,place gone hill clearli cut staff food qualiti...,0.0,NaN,0.0,0.0
4,kbtscdyz6lvrtGjD1quQTg,4,like walk time saturdai morn sister bowl leagu...,NaN,NaN,NaN,1.0


In [7]:
# extract all review ids that metions each of the four categories
def extract_sentiment(category):  
    reviews = []
    for i in range(len(df2[category])):
        for j in range(len(sentiment_df)):
            if df2['review_id'][i] == sentiment_df['review_id'][j] and df2[category][i] == 1:
                reviews.append(sentiment_df.iloc[j])
    print(len(reviews))  
    return reviews

In [9]:
categories = ['food', 'ambience', 'service', 'price']
food_df = pd.DataFrame(extract_sentiment(categories[0]))
food_df
ambience_df = pd.DataFrame(extract_sentiment(categories[1]))
ambience_df
service_df = pd.DataFrame(extract_sentiment(categories[2]))
service_df
price_df = pd.DataFrame(extract_sentiment(categories[3]))
price_df

189
83
137
55


,review_id,stars,text,food_pos,ambience_pos,service_pos,price_pos
243,TFfoC8UJzvQ96abwA7I9wQ,5,good clean place mani pedi came tax great job ...,1.0,0.0,1.0,0.0
49,4PKiLkFTff9HK44QkOo6vA,4,heard place dy check glad decent beer select k...,1.0,0.0,NaN,1.0
275,OED6UmN9WWtBrGCfpokffg,3,tricki cool look spot nice decor ambianc comfi...,1.0,1.0,0.0,0.0
174,V93SYj2OLh5m9Cquzf-7kg,4,came town countri concert better wai start dai...,1.0,1.0,1.0,1.0
247,ILQO7XB-EJeqjaZSM9MHsw,5,let shock peopl super friendli food donut clos...,1.0,0.0,1.0,1.0
56,m2H0m3pDUNmkXZP4K8c_oA,2,advertis vegetarian dinner price paid like nic...,0.0,NaN,1.0,0.0
83,_LPOtmfWnL_8_tL3ImhoLw,5,sure post neg review place point girlfriend di...,1.0,0.0,1.0,1.0
282,O_w2fnNmG-zJcbN8ft5DAw,5,paco taco real deal spite great yelp review re...,1.0,1.0,1.0,1.0
267,fUiAHODVCD2f1tVvwFe6Fg,5,belizean food heard right build month final wa...,1.0,NaN,1.0,1.0
365,-61dxd2EWpVAwxRFv7zJgQ,5,came lunch person includ drink huge select fla...,1.0,0.0,0.0,1.0


In [11]:
train, test = train_test_split(food_df, test_size=0.20, shuffle=True)
X_train = train.text
X_test = test.text
print(X_train.shape)
print(X_test.shape)

(151,)
(38,)


In [12]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(train['text'])
test_vectors = vectorizer.transform(test['text'])

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, train['food_pos'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
# report = classification_report(test['food_pos'], prediction_linear, output_dict=True)
print(classification_report(test['food_pos'], prediction_linear))

Training time: 0.010392s; Prediction time: 0.001786s
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         7
         1.0       0.82      1.00      0.90        31

    accuracy                           0.82        38
   macro avg       0.41      0.50      0.45        38
weighted avg       0.67      0.82      0.73        38



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
train2, test2 = train_test_split(ambience_df, test_size=0.20, shuffle=True)
X_train2 = train2.text
X_test2 = test2.text
print(X_train2.shape)
print(X_test2.shape)

train2.sample(frac=1).head(5)

# Create feature vectors
vectorizer2 = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train2_vectors = vectorizer.fit_transform(train2['text'])
test2_vectors = vectorizer.transform(test2['text'])

# Perform classification with SVM, kernel=linear
classifier_linear2 = svm.SVC(kernel='linear')
t0_2 = time.time()
classifier_linear2.fit(train2_vectors, train2['ambience_pos'])
t1_2 = time.time()
prediction_linear2 = classifier_linear2.predict(test2_vectors)
t2_2 = time.time()
time_linear_train2 = t1_2-t0_2
time_linear_predict2 = t2_2-t1_2
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train2, time_linear_predict2))
print(classification_report(test2['ambience_pos'], prediction_linear2))

(66,)
(17,)
Training time: 0.007605s; Prediction time: 0.000710s
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.82      1.00      0.90        14

    accuracy                           0.82        17
   macro avg       0.41      0.50      0.45        17
weighted avg       0.68      0.82      0.74        17



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
train3, test3 = train_test_split(service_df, test_size=0.20, shuffle=True)
X_train3 = train3.text
X_test3 = test3.text
print(X_train3.shape)
print(X_test3.shape)

train3.sample(frac=1).head(5)

# Create feature vectors
vectorizer3 = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train3_vectors = vectorizer.fit_transform(train3['text'])
test3_vectors = vectorizer.transform(test3['text'])

# Perform classification with SVM, kernel=linear
classifier_linear3 = svm.SVC(kernel='linear')
t0_3 = time.time()
classifier_linear3.fit(train3_vectors, train3['service_pos'])
t1_3 = time.time()
prediction_linear3 = classifier_linear3.predict(test3_vectors)
t2_3 = time.time()
time_linear_train3 = t1_3-t0_3
time_linear_predict3 = t2_3-t1_3
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train3, time_linear_predict3))
print(classification_report(test3['service_pos'], prediction_linear3))

(109,)
(28,)
Training time: 0.004482s; Prediction time: 0.000894s
              precision    recall  f1-score   support

         0.0       0.60      0.43      0.50         7
         1.0       0.83      0.90      0.86        21

    accuracy                           0.79        28
   macro avg       0.71      0.67      0.68        28
weighted avg       0.77      0.79      0.77        28



In [15]:
train4, test4 = train_test_split(price_df, test_size=0.20, shuffle=True)
X_train4 = train4.text
X_test4 = test4.text
print(X_train4.shape)
print(X_test4.shape)

train4.sample(frac=1).head(5)

# Create feature vectors
vectorizer4 = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train4_vectors = vectorizer.fit_transform(train4['text'])
test4_vectors = vectorizer.transform(test4['text'])

# Perform classification with SVM, kernel=linear
classifier_linear4 = svm.SVC(kernel='linear')
t0_4 = time.time()
classifier_linear4.fit(train4_vectors, train4['price_pos'])
t1_4 = time.time()
prediction_linear4 = classifier_linear4.predict(test4_vectors)
t2_4 = time.time()
time_linear_train4 = t1_4-t0_4
time_linear_predict4 = t2_4-t1_4
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train4, time_linear_predict4))
print(classification_report(test4['price_pos'], prediction_linear4))

(44,)
(11,)
Training time: 0.003412s; Prediction time: 0.000343s
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.70      0.88      0.78         8

    accuracy                           0.64        11
   macro avg       0.35      0.44      0.39        11
weighted avg       0.51      0.64      0.57        11

